## Introduction to Scores, Risk, Alphas and Holdings

Some of the principles below are similar to those applied by professional research teams. We are
going to use open source tools only and therefore will need to keep the examples simple.

### Raw signal to scores

Scores are build on a raw signal. Raw Signals – Any raw data used to predict future returns
e.g. past stock returns, survey values, interest rates, earnings estimate, buy/sell recommendation.
Features of raw signals: - They are not a direct forecast of exceptional return - They have a variety
of units and scales
Real signal data is noisy, contains the true value + error. We can’t observe the true value or
attempt to forecast it from noisy data.
Exponential Moving Average – EMA is used below to smooth and standardize the raw signal.
We could apply two types of scoring (or even both). - Time Series Scoring. Current score of an
asset is relative to its own history in the past. Is the option relatively cheaper or more expensive
compared to historical values? - Cross Sectional Scoring. Current score of an asset is relative to
its asset peers today. Is the option more or less expensive compared to other stocks in the same
sector?
Below we are reading the ‘raw signal’ from pack 1.

In [112]:
import os
import pickle
import pandas as pd
import numpy as np
import math as m
import matplotlib
import matplotlib.pyplot as plt

In [113]:
def xscore(df):
    """ Cross sectionally score/standardize two or three dimensional timeseries"""
    return df.sub(df.mean(axis=1), axis=0).div(df.std(axis=1), axis=0)

def tscore(df, halflife):
    """Standardizes the input
    - Returns:
    timeseries of scores, means and volatility
    """
    dfmean = df.ewm(halflife=halflife).mean()
    dfstd = df.ewm(halflife=halflife).std()
    return (df - dfmean) / dfstd, dfmean, dfstd



In [114]:
data_root = r'D:\Algothonslack\Packs'
print('Read returns and raw signal')
# read previously calculated signal forecast
forecasts = pd.read_csv(os.path.join(data_root,'forecast_example_data.csv')).set_index('period')
# read the actual returns of the assets
returns = pd.read_csv(os.path.join(data_root, 'returns_example_data.csv')).set_index('period')
print('Calculating scores and volatilities')
forecasts.index = forecasts.index.astype(pd.datetime)
returns.index = returns.index.astype(pd.datetime)
# halflife relates to the speed fo the signal and requires some additional 
tscores, tmean, tstd = tscore(forecasts, halflife=52)
# for xscoring, vol scale the signals first
volScaled = forecasts / tstd
xscores = xscore(volScaled)

Read returns and raw signal
Calculating scores and volatilities


### Ensure no peak ahead

We do this by shifting the scores by at least one time period. That means that the scores we
calculate with data from today can be used to predict returns only for the following date

In [115]:
# lag the signals to avoid peekahead
tscores = tscores.shift(1)
xscores = xscores.shift(1)

### Generate the risk data

In [116]:
print('Calculating risk')
# Estimating the risk through a rolling 22 day sample and annualising.
# The rolling window here can be set according to what makes sense for the # Since it take a bit to generate, you could calculate it once and then
# read it from disk
use_cached_risk = True
risk_file = os.path.join(data_root,'risk.pickle')
if os.path.exists(risk_file) and use_cached_risk:
    with open(risk_file, 'rb') as handle:
        riskMx = pickle.load(handle)
else:
    riskMx = returns.rolling(22).cov() * 252
    with open(risk_file, 'wb') as handle:
        pickle.dump(riskMx, handle, protocol=pickle.HIGHEST_PROTOCOL)

Calculating risk


### Generating Alphas and Holdings

First we convert scores to alphas using the Grinold-Khan formula. where the IC is the information
coefficient and sigma is the signals risk. The IC can be derived or set as a constant and it represents
a portfolio manager’s ‘skill’ in selecting securities. A default value of IC can be set to 0.05.

$\alpha = $ score $* \mbox{ IC }* \sigma$

The Grinold-Khan formula is only one way to generate the alphas, other methods exist including
discretionary views alphas which come from specialist internal or external surveys.
Then we calculate the holdings by maximing the utility function in the absence of any constraints.
The lamda here is a constant representing risk aversion which we can set to 1. This is
one of the simplest ways to run an optimization of asset alphas in relation to their risk and the
investor’s risk aversion.

$U = \alpha h - \lambda V h^2$

$\frac{\partial U}{\partial h} = \alpha - 2 \lambda V h$


$ h = \frac{\alpha V^{-1}}{2\lambda}$

In [117]:
def gen_alpha_and_holdings(score, vols, ic=0.05):
    """Compute alphas from scores
    Args:
    score: Z scores
    ic: Information coeffcient
    vols: Asset volatilites
    Returns:
    Alphas (i.e. alpha = IC * score * volatility)
    """
    alphas = pd.DataFrame()
    holdings = pd.DataFrame()
    vol_data = vols.values.reshape(int(len(vols.values) / len(vols.columns)),len(vols.columns),len(vols.columns))
                                   
    for ind in range(len(score.index)):
        currRiskMx = np.array(vol_data[ind, :,:])
        if not np.isnan(currRiskMx).all():
            invmx = np.linalg.inv(currRiskMx)
            alpha = tscores.iloc[ind, :] * ic * np.diag(currRiskMx) ** 0.5
            alphas = alphas.append(alpha)
            holding_data = alpha.dot(invmx) / 2
            holding_series = pd.Series(holding_data, index=alpha.index)
            holding_series.name = alpha.name
            holdings = holdings.append(holding_series)
    return alphas, holdings

In [119]:
# Generating alphas and holdings
alphas, holdings = gen_alpha_and_holdings(xscores, riskMx)
alphas.to_csv(os.path.join(data_root,'alphas.csv'))
holdings.to_csv(os.path.join(data_root, 'holdings.csv'))

### Backtesting of Holdings

There are many metrics to be calculated once the model holdings have been generated. For this
simple example we will start with Information Ratio (IR) and Turnover.
IR is the risk-adjusted rate of return. IR can be used not only on individual assets or the
model as a whole but also on holdings generated from differently lagged scores or with different
combinations of assets which can test the robustness of the signal.
Turnover represents the percentage of the portfolio holdings that have changed over a time
period. A high turnover signal can be problematic when used on non highly liquid assets or when
the tcost and/or market impact are high. This suggests that the signal’s trading frequency, assets,
optimization or tcost model should be reviewed.

In [120]:
def freq_str(df):
    """ Returns the inferred frequency of the timeseries
    """
    if df.index.inferred_freq is None:
        return
    inferred_freq = df.index.inferred_freq.split('-')[0]
    if inferred_freq in ('B', 'D'): # daily
        return 'daily'
    elif inferred_freq == 'W': # weekly
        return 'weekly'
    elif inferred_freq in ('M', 'BM', 'MS', 'BMS'): # monthly
        return 'monthly'
    elif inferred_freq in ('Q', 'BQ', 'QS', 'BQS'): # quarterly
        return 'quarterly'
    elif inferred_freq in ('A', 'BA', 'AS', 'BAS'): # annual
        return 'annual'
    elif inferred_freq == 'H': # hourly
        return 'hourly'
    elif inferred_freq == 'T': # minutely
        return 'minutely'
    elif inferred_freq == 'S': # secondly
        return 'secondly'
    elif inferred_freq == 'L': # millisecondly
        return 'millisecondly'
    elif inferred_freq == 'U': # microsecondly
        return 'microsecondly'
    else:
        annual_freq = round(freq(df))
        if annual_freq == 1:
            return 'annual'
        elif annual_freq == 4:
            return 'quarterly'
        elif annual_freq == 12:
            return 'monthly'
        elif annual_freq == 52:
            return 'weekly'
        elif 250 <= annual_freq <= 365:
            return 'daily'

In [121]:
freq_lookup = {'annual': 1, 'quarterly': 4, 'monthly': 12, 'weekly': 52, 'hourly': 260 * 24, 'minutely': 260 * 24 * 60,  'microsecondly': 260 * 24 * 60 * 1e6}
               

In [134]:
def freq(df, inferred=False):
    """Compute frequency of timeseries in units of periods per year
    Args:
    df: timeseries
    inferred (bool): Infer the frequency (default is True). If True return annual: 1
    quarterly: 4
    monthly: 12
    weekly: 52
    daily: 260
    """
    #empirical_freq = (df.shape[0] - 1) / (df.index[-1].days - df.index[0].days)
    if inferred:
        s = freq_str(df)
        if s in freq_lookup:
            return freq_lookup[s]
        else:
            log.warning('Can not infer frequency')
    else:
        # default to daily frequency
        return 260 

In [135]:
def ir(df):
    """Compute realized information ratio
    """
    mean = df.mean()
    stdev = df.std()
    annualization_factor = np.sqrt(freq(df))
    if isinstance(stdev, pd.Series):
        stdev.replace(0, np.nan, inplace=True)
    elif stdev == 0:
        stdev = np.nan
    return mean / stdev * annualization_factor

In [136]:
def turnover(h, ret=None):
    """Compute average portfolio turnover
    Args:
    h: Holdings
    ret: Optional asset returns used to adjust initial holdings
    Returns:
    Annualized absolute portfolio turnover
    """
    h = h.fillna(0.0)
    if ret is None:
        h_initial = h.shift()
    else:
        h_initial = h.mul(1 + ret).div(1 + h.mul(ret).sum(axis=1), axis=0)
    trades = h - h_initial
    return trades.abs().sum(axis=1).mean() * freq(h)

In [137]:
print('calculate IR')
print(ir(holdings))
print(' ##### ')
print('calculate Turnover')
print(turnover(holdings,returns))

calculate IR
X0000   -0.807784
X0001    0.176091
X0002   -1.444355
X0003   -0.517490
X0004   -0.130676
X0005    0.639562
X0006    1.763498
X0007    0.943258
X0008   -0.448178
X0009    0.084749
X0010    0.152479
X0011    1.015571
X0012    1.305033
X0013    0.483378
X0014    1.500183
X0015   -0.316819
X0016    0.272968
X0017    1.385198
X0018   -0.097904
X0019    0.315289
X0020    1.675392
X0021   -1.257452
X0022    1.351893
X0023   -0.963739
X0024    1.836852
X0025   -1.775731
X0026    0.829655
X0027    0.722068
X0028    1.112634
X0029    1.355711
           ...   
X0070    1.524877
X0071    1.548724
X0072   -0.829707
X0073    0.934761
X0074   -0.197428
X0075   -0.221506
X0076    1.019060
X0077   -0.112239
X0078    0.414770
X0079   -0.606003
X0080   -1.926062
X0081   -0.222489
X0082   -0.732526
X0083   -1.524440
X0084    1.537653
X0085    1.404291
X0086    0.320020
X0087   -0.575189
X0088    0.365376
X0089   -1.315264
X0090   -1.361866
X0091   -0.692478
X0092   -1.319289
X0093   -1.6312